In [ ]:
import json
import telebot
from openai import OpenAI
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader

import os

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
import time

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# 以上都是乱七八糟的import

In [ ]:
import os
os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

## 第三方OpenAI API配置

In [ ]:
from openai import OpenAI

api_key = ""
api_base = ""
client = OpenAI(api_key=api_key, base_url=api_base)

dialogue_model = "gpt-4o-mini" # "gpt-3.5-turbo"
embed_model = 'text-embedding-3-large'

In [ ]:
# test
import json

messages_model = [{"role": "user", "content": "我正在寻求获取自2020年起发表的学术论文，并特别希望审阅这些论文中的特定两页内容。"}]
response = client.chat.completions.create(model=dialogue_model, messages=messages_model, tools=tools, tool_choice={"type": "function", "function": {"name": "extract_as_ylo_start_from_user_request"}})
print(response.choices[0].message)
as_ylo = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["as_ylo"]
start = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["start"]
print(as_ylo, start)

## ChatBot配置

In [ ]:
import telebot

BOT_TOKEN = ""
bot = telebot.TeleBot(BOT_TOKEN)

## 消息处理器

### 1. 通配符处理器 - 不是命令的请求都是用api调用model进行回答

#### 整合从数据库中查询到的信息和原始问题

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_query_info",
            "description": "判断用户的请求属于哪一个领域的问题，并适当修改用户的请求，使提高在向量数据库中检索的准确度。",
            "parameters": {
                "type": "object",
                "properties": {
                    "domain": {
                        "type": "string",
                        "enum": ["中医", "计算机", "文学", "None"],
                        "description": "用户的请求属于的领域。仅允许从以下四个领域中选择一个：1.中医，2.计算机，3.文学，4.None。"
                    },
                    "content": {
                        "type": "string",
                        "description": "修改后的用户请求，修改的目的是为了提高在向量数据库中检索的准确度。"
                    }
                },
                "required": ["domain", "content"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_as_ylo_start_from_user_request",
            "description": "判断用户需要查询从几几年开始的文章和需要查询几页数据。",
            "parameters": {
                "type": "object",
                "properties": {
                    "as_ylo": {
                        "type": "string",
                        "description": "用户需要查询从几几年开始的文章。 (e.g., 2018, 2021)"
                    },
                    "start": {
                        "type": "string",
                        "description": "用户需要查询几页的数据。 (e.g., 1, 3)"
                    }
                },
                "required": ["as_ylo", "start"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_keywords_from_user_request",
            "description": "从用户请求中提取关键字以便进行爬取文章。",
            "parameters": {
                "type": "object",
                "properties": {
                    "q": {
                        "type": "string",
                        "description": "从用户请求中提取关键字方便后续使用爬虫爬取相关文章，如果关键字多于一个，则使用“+”进行连接，只允许出现英文字母或中文字，不要产生空格。 (e.g., PE, PE+CNN)"
                    }
                },
                "required": ["q"]
            }
        }
    }
]

In [ ]:
# test
messages = [{"role": "user", "content": "What is a Gaussian process?"}]
response = client.chat.completions.create(model=dialogue_model, messages=messages,
                                          tools=tools, tool_choice={"type": "function", "function": {"name": "extract_query_info"}})
print(response.choices[0].message)

In [ ]:
# test
import json

json.loads(response.choices[0].message.tool_calls[0].function.arguments)["domain"], json.loads(response.choices[0].message.tool_calls[0].function.arguments)["content"]

In [ ]:
def QueryinSpecifiedDatabase(domain, content, top_k=3):
    """
    domain：根据domain判断比对哪一个数据库
    content：用户请求的修改版本
    top_k：返回前k个最相似的文本
    """
    content_embeddings = client.embeddings.create(input=content, model=embed_model).data[0].embedding 

    if domain == '中医':
        namespace = "TraditionalChineseMedicine_DataBase"
    elif domain == '计算机':
        namespace = "CS_DataBase"
    elif domain == '文学':
        pass
    elif domain == 'None':
        pass

    results = index.query(
        namespace = namespace,
        vector = content_embeddings,
        top_k = top_k,
        include_values = True,
        include_metadata = True
    )

    results_list = [result['metadata']['text'] for result in results['matches']]
    return results_list 

In [ ]:
def GetSystemPrompt(domain):
    if domain == '中医':
        return "你是一位经验丰富的中医，擅长使用中医知识来解决问题。"
    elif domain == '计算机':
        return "你是一位精通计算机技术的专家，你精通的技术包括但不限于机器学习，深度学习，各种编程语言，各种API的调用。"
    elif domain == '文学':
        return ""
    elif domain == 'None':
        return ""

In [ ]:
def construct_prompt(similar_info, question):
    """
    similar_info：从指定数据库中查询到的有关信息，一个list
    question：原始问题
    """
    prompt = "以下是我们从向量数据中查询到的与问题相关的背景信息：\n"
    for i, info in enumerate(similar_info, 1):
        prompt += f"背景信息 {i}: {info}\n"
    prompt += f"\n基于上述信息，请回答以下问题：\n{question}\n"
    return prompt

In [ ]:
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "rag-chatbot"
index = pc.Index(index_name)

In [ ]:
import json

# 确保"通配符处理器"不会处理处理以斜杠"/"开头的命令
@bot.message_handler(func=lambda message: not message.text.startswith('/'))
def echo_all(message): # bot.reply_to(message, message.text) # 回声机器人
    response = client.chat.completions.create(model=dialogue_model, messages=[{"role": "user", "content": message.text}], 
                                              tools=tools, tool_choice={"type": "function", "function": {"name": "extract_query_info"}})
    domain = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["domain"]
    content = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["content"]

    if domain == "None": # 如果用户的请求不是任意一个特定的领域，则直接调用model进行回答
        completion = client.chat.completions.create(
            model=dialogue_model,
            messages=[
                {"role": "system", "content": "You are a helpful AI assiatant that helps with any questions. You need to be as concise as possible in your answer."},
                {"role": "user", "content": message.text}
            ]
        )
        bot.reply_to(message, completion.choices[0].message.content)
    else:
        if domain == '中医':
            bot.send_message(message.chat.id, "正在调用TraditionalChineseMedicine_DataBase，请稍后！")
        elif domain == '计算机':
            bot.send_message(message.chat.id, "正在调用CS_DataBase，请稍后！")
            
        # 从指定数据库中匹配最相似的信息
        InformationfromSecifiedDatabase = QueryinSpecifiedDatabase(domain, content) # 返回的是list
        bot.send_message(message.chat.id, InformationfromSecifiedDatabase)

        prompt = construct_prompt(InformationfromSecifiedDatabase, message.text)
        bot.send_message(message.chat.id, prompt)

        system_prompt = GetSystemPrompt(domain)
        # bot.send_message(message.chat.id, system_prompt)
    
        completion = client.chat.completions.create(
            model=dialogue_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
    
        bot.reply_to(message, completion.choices[0].message.content)

### 2. 命令处理器

#### (1) 开始聊天

In [ ]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Hi, I am a smart chatbot developed by Cui Peng, how can I do for you?")

#### (2) 利用API的运势预测器

In [ ]:
@bot.message_handler(commands=['horoscope'])
def sign_handler(message):
    text = "你的星座是什么？\n选一个: *Aries*, *Taurus*, *Gemini*, *Cancer,* *Leo*, *Virgo*, *Libra*, *Scorpio*, *Sagittarius*, *Capricorn*, *Aquarius*, and *Pisces*."
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, day_handler)

def day_handler(message):
    sign = message.text # 星座
    text = "你想知道哪天的呀？\n选一个吧: *TODAY*, *TOMORROW*, *YESTERDAY*, 或其他 YYYY-MM-DD 格式的日期。"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, fetch_horoscope, sign.capitalize())

def fetch_horoscope(message, sign): # 输入：哪一天，星座
    day = message.text
    messages_to_openaimodel=[
        {"role": "system", "content": "你是一个擅长星座运势预测的AI机器人。我将提供我的星座和我需要预测的日期，请根据我提供的星座和日期进行运势预测。"},
        {"role": "user", "content": "我的星座是："+sign+" "+"我需要预测的日期是："+day}
    ]
    bot.send_message(message.chat.id, messages_to_openaimodel[1]["content"])
    response = client.chat.completions.create(model=dialogue_model, messages=messages_to_openaimodel)
    bot.send_message(message.chat.id, "你的运势来啦!")
    bot.send_message(message.chat.id, response.choices[0].message.content, parse_mode="Markdown")

#### (3) 利用API的爬虫 - 构建RAG数据库

In [ ]:
from request_processor import request_processor

In [ ]:
# test
user_request = "How to treat acne?"
predicted_label, predicted_prob = request_processor(user_request)
print(type(predicted_label), type(predicted_prob), f"Predicted label: {predicted_label}, Probability: {predicted_prob}")

In [ ]:
@bot.message_handler(commands=['crawl'])
def initiate_crawl(message):
    text = "您需要爬取哪方面的文章？注：请使用英文进行提问，中文提示会报错！"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, process_query)

def process_query(message): 
    user_request = message.text # 用户请求类似 "What is Foucault's philosophy?"
    
    predicted_label, predicted_prob = request_processor(user_request)
    predicted_label, predicted_prob = str(predicted_label), str(predicted_prob)
    
    text = "文本分类模型预测您需要查询的领域为"+predicted_label+"，模型认为您的请求属于该领域的概率为"+predicted_prob+"。\n如果模型的预测结果正确，请输入：T，否则请输入：F。"
    
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, validate_prediction, user_request, predicted_label)

# 验证文本分类模型是否正确分类
def validate_prediction(message, user_request, predicted_label):
    message_text = message.text  # 输入为：T 或 F
    
    if message_text == "T": 
        text = "我们的文本分类模型真是一个好模型！请问您需要爬取从几几年开始的文章？您需要查看前多少页的文章（一页有10篇文章）？"
        sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
        true_label = predicted_label
        bot.register_next_step_handler(sent_msg, get_crawl_details, user_request, true_label)
    elif message_text == "F":  
        text = "非常抱歉造成错误，我们将会记录您的本次提问，这将会有助于提高我们文本分类模型的准确率。\n您的请求属于education，CS，medical，literature，other中的哪一个？"
        sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
        bot.register_next_step_handler(sent_msg, handle_incorrect_classification, user_request, predicted_label)

In [ ]:
from AppendRowtoCSV import append_row_to_csv

def handle_incorrect_classification(message, user_request, predicted_label):
    true_label = message.text # 输入为education，CS，medical，literature，other
    
    csv_file = f'data/data_misclassified_user_requests.csv'
    new_row = {'Text': user_request, 'Label_true': true_label, "Label_predictedbyModel": predicted_label}
    append_row_to_csv(csv_file, new_row)

    text = "非常抱歉造成错误，我们已经记录了本次错误信息，在此向您表示我们诚挚的歉意！请问您需要爬取从几几年开始的文章？您需要查看前多少页的文章（一页有10篇文章）？"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, get_crawl_details, user_request, true_label)

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import List

response_format_extract_as_ylo_and_start = {
    "type": "json_schema",
    "json_schema": {
        "name": "extract_as_ylo_and_start",
        "schema": {
            "type": "object",
            "properties": {
                "as_ylo_for_searching": {
                    "type": "string",
                    "description": "从用户请求中提取的年份, for example: User request: 我正在寻求获取自2020年起发表的学术论文，并特别希望审阅这些论文中的特定两页内容。 - Extracted as_ylo: 2020"
                },
                "start_for_searching": {
                    "type": "string",
                    "description": "从用户请求中提取的需要查找几页的内容, for example: User request: 我正在寻求获取自2020年起发表的学术论文，并特别希望审阅这些论文中的特定两页内容。 - Extracted start: 2"
                } 
            },
            "required": ["as_ylo_for_searching", "start_for_searching"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class As_Ylo_and_Start(BaseModel):
    as_ylo_for_searching: str
    start_for_searching: str

response_format_extract_keywords = {
    "type": "json_schema",
    "json_schema": {
        "name": "extract_keywords",
        "schema": {
            "type": "object",
            "properties": {
                "keywords_for_searching": {
                    "type": "string",
                    "description": "Keywords used for query extracted from user requests, for example: 1. User request: Can CNN be combined with PE? - Extracted keywords: CNN+PE; 2. User request: How to treat ulcers? - Extracted keywords: ulcer treatment."
                }
            },
            "required": ["keywords_for_searching"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class Keywords(BaseModel):
    keywords_for_searching: str

In [ ]:
client = OpenAI() # json格式暂时必须使用OPENAI官方的API

In [ ]:
import json

from crawler.GoogleScholar import fetch_and_save_results_temp

def get_crawl_details(message, user_request, label):
    """
    message.text：类似 “我需要查找从2021年开始的文章，并且需要查看前5页的文章？”
    user_request：类似："What is Foucault's philosophy?"
    label：one of "education，CS，medical，literature，other"
    """
    message_text = message.text 

    # 使用openai结构化输出提取as_ylo和start
    messages_model = [{"role": "system", "content": "You are a helpful bot that helps me extract specific information from user requests."},
                      {"role": "user", "content": message_text}]
    # response = client.chat.completions.create(model=dialogue_model, messages=messages_model, tools=tools, tool_choice={"type": "function", "function": {"name": "extract_as_ylo_start_from_user_request"}})
    # as_ylo = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["as_ylo"]
    # start = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["start"]
    response = client.chat.completions.create(model=dialogue_model, messages=messages_model, response_format=response_format_extract_as_ylo_and_start)
    try: # 看看response.choices[0].message.content是不是我们期待的json格式
        as_ylo_and_start = As_Ylo_and_Start.parse_raw(response.choices[0].message.content)
        as_ylo_and_starts_dict = as_ylo_and_start.dict()
        as_ylo = as_ylo_and_starts_dict["as_ylo_for_searching"]
        start = as_ylo_and_starts_dict["start_for_searching"]
    except ValidationError as e:
        # Handle validation errors
        print(e.json())
        bot.send_message(message.chat.id, "解析开始年份和页数时出错，请重新尝试。")
        return
    
    # 使用openai结构化输出提取keywords
    messages_model = [{"role": "system", "content": "You are a helpful bot that helps me extract specific information from user requests."},
                      {"role": "user", "content": user_request}]
    # response = client.chat.completions.create(model=dialogue_model, messages=messages_model, tools=tools, tool_choice={"type": "function", "function": {"name": "extract_keywords_from_user_request"}})
    # keyword = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["q"]
    response = client.chat.completions.create(model=dialogue_model, messages=messages_model, response_format=response_format_extract_keywords)
    try:
        keywords = Keywords.parse_raw(response.choices[0].message.content)
        keywords_dict = keywords.dict()
        keywords = keywords_dict["keywords_for_searching"]
    except ValidationError as e:
        # Handle validation errors
        print(e.json())
        bot.send_message(message.chat.id, "解析关键字时出错，请重新尝试。")
        return

    # print(type(keywords), type(as_ylo), type(start))
    text = f"关键字为：{keywords}，开始年份为：{as_ylo}，爬取的页数为：{start}。开始爬取！请稍后。"

    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")

    # ######################################################################################
    # # 由于调用openai tools不一定有效所以我们手动处理一下keyword
    keywords = clean_string(keywords)
    # # 调用爬取函数之前的准备
    temp_save_path = f"data/data_crawled/database_temporary/{label}.csv"
    # # 调用爬取的函数
    df_crawled = fetch_and_save_results_temp(temp_save_path, keywords, as_ylo, int(start))
    text = "爬取完成，请开始后续的提问！"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    # ######################################################################################

    # # bot.register_next_step_handler(sent_msg, 爬取, keyword, as_ylo, start)

In [ ]:
import re  

def clean_string(input_string):  
    # 使用正则表达式来匹配所有英文字母和中文字符  
    cleaned_string = re.sub(r'[^a-zA-Z\u4e00-\u9fa5]', '', input_string)  
    return cleaned_string 

# 示例使用  
# input_str = "Hello + 世界 123! 你好   "  
# result = clean_string(input_str)  
# print(result)  # 输出: Hello世界你好  

In [ ]:
from crawler.GoogleScholar import 

# 调用爬取的函数
def 爬取(keyword, keyword, start):
    """
    keyword
    as_ylo
    start
    """
    text = "请问您需要查询从几几年开始的文章？我查询多少篇文章？"

#### (4) 小彩蛋

In [ ]:
@bot.message_handler(commands=['love'])
def lover_first(message):
    text = "猜猜机器人的作者喜欢谁"
    sent_msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(sent_msg, lover_handler)

def lover_handler(message):
    if message.text == "沈心怡":
        bot.send_message(message.chat.id, "恭喜你！回答正确！")
    else:
        bot.send_message(message.chat.id, "回答错误哦，请再想一想。")
        sent_msg = bot.send_message(message.chat.id, "猜猜机器人的作者喜欢谁")
        bot.register_next_step_handler(sent_msg, lover_handler)

## 启动ChatBot

In [ ]:
bot.infinity_polling()